In [1]:
%matplotlib inline 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
import numpy as np 

In [2]:
df = pd.read_csv("../Datasets/Churn_Modelling_21_14.csv")
df.shape

(10000, 14)

In [3]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df_clean = df.drop(['CustomerId','Surname'], axis='columns')
df_clean.columns

Index(['RowNumber', 'CreditScore', 'Geography', 'Gender', 'Age', 'Tenure',
       'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Exited'],
      dtype='object')

In [6]:
df_clean.dtypes

RowNumber            int64
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [7]:
df_clean= pd.get_dummies(df_clean, ['Geography','Gender'],drop_first=True)
df_clean.columns

Index(['RowNumber', 'CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'Geography_Germany', 'Geography_Spain', 'Gender_Male'],
      dtype='object')

In [8]:
df_clean.head()

,RowNumber,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,1,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,2,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,3,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,4,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,5,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [9]:
scalar = MinMaxScaler()
df_clean[['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']] = scalar.fit_transform(df_clean[['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']])

In [10]:
df_clean.head()

,RowNumber,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,1,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,0,0,0
1,2,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,1,0
2,3,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,0,0,0
3,4,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,0,0,0
4,5,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,1,0


In [11]:
def ANN(X_train,y_train,X_test,y_test,loss,weights, epochs):
    model = keras.Sequential ([ 
        keras.layers.Dense(12, input_dim=12,activation='relu' ),
        keras.layers.Dense(8,activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
     ])

    model.compile(optimizer = 'adam', loss=loss, metrics=['accuracy'])

    if weights == -1:
        model.fit(X_train,y_train,epochs=epochs)
    else:
        model.fit(X_train,y_train, class_weight = weights, epochs=epochs)
    
    print(model.evaluate(X_test,y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print(f'classification report: \n ', classification_report(y_test, y_preds))

    return y_preds

# Step #1 create a input dataset to be later used for undersampling, oversampling ..

In [12]:
# store churn variation counts in two variables 
count_exited_0, count_exited_1 = df_clean.Exited.value_counts()

count_exited_0 , count_exited_1

(7963, 2037)

In [13]:
df_exited_0 = df_clean[df_clean.Exited==0]
df_exited_1 = df_clean[df_clean.Exited==1]

df_exited_0.shape , df_exited_1.shape

((7963, 13), (2037, 13))

In [14]:
X = df_clean.drop(['Exited'], axis='columns')
y = df_clean.Exited
X.shape, y.shape

((10000, 12), (10000,))

# Create DS for standard model

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=15,stratify=y)
X_train.shape,X_test.shape

((8000, 12), (2000, 12))

In [16]:
y_train.value_counts() , y_test.value_counts()

(0    6370
 1    1630
 Name: Exited, dtype: int64,
 0    1593
 1     407
 Name: Exited, dtype: int64)

In [17]:
y_pred = ANN(X_train,y_train,X_test,y_test,'binary_crossentropy',-1,20)


Epoch 1/20
250/250 [==============================] - 0s 741us/step - loss: 0.7798 - accuracy: 0.6963
Epoch 2/20
250/250 [==============================] - 0s 744us/step - loss: 0.8742 - accuracy: 0.7179
Epoch 3/20
250/250 [==============================] - 0s 771us/step - loss: 0.6694 - accuracy: 0.7544
Epoch 4/20
250/250 [==============================] - 0s 763us/step - loss: 0.6541 - accuracy: 0.7524
Epoch 5/20
250/250 [==============================] - 0s 768us/step - loss: 0.8064 - accuracy: 0.7372
Epoch 6/20
250/250 [==============================] - 0s 722us/step - loss: 0.7485 - accuracy: 0.7431
Epoch 7/20
250/250 [==============================] - 0s 719us/step - loss: 0.7307 - accuracy: 0.7491
Epoch 8/20
250/250 [==============================] - 0s 748us/step - loss: 0.8371 - accuracy: 0.7339
Epoch 9/20
250/250 [==============================] - 0s 748us/step - loss: 0.6303 - accuracy: 0.7623
Epoch 10/20
250/250 [==============================] - 0s 703us/step - loss: 0.798

# Method 1: Model for undersampling

## Exited customer are less in numbers in DS vs Customers still active - undersampling 

In [18]:
df_consol = X_train.copy()
df_consol.shape

(8000, 12)

In [19]:
df_consol['Exited'] = y_train
df_consol.shape

(8000, 13)

In [20]:
count_exited_0 = df_consol[df_consol.Exited==0]
count_exited_1 = df_consol[df_consol.Exited==1]
count_exited_0.Exited.value_counts(), count_exited_1.Exited.value_counts()

(0    6370
 Name: Exited, dtype: int64,
 1    1630
 Name: Exited, dtype: int64)

In [21]:
count_exited_0_under = count_exited_0.sample(1630)
count_exited_0_under.shape

(1630, 13)

In [22]:
df_under_input = pd.concat([count_exited_0_under,count_exited_1],axis=0)
df_under_input.shape

(3260, 13)

In [23]:
X= df_under_input.drop(['Exited'],axis='columns')
y=df_under_input.Exited
X.shape, y.shape

((3260, 12), (3260,))

In [24]:
y_pred_under =  ANN(X,y,X_test,y_test,'binary_crossentropy',-1,20)

Epoch 1/20
102/102 [==============================] - 0s 932us/step - loss: 214.6073 - accuracy: 0.5058
Epoch 2/20
102/102 [==============================] - 0s 898us/step - loss: 1.2712 - accuracy: 0.5135
Epoch 3/20
102/102 [==============================] - 0s 1ms/step - loss: 0.7889 - accuracy: 0.5049
Epoch 4/20
102/102 [==============================] - 0s 1ms/step - loss: 0.7319 - accuracy: 0.5126
Epoch 5/20
102/102 [==============================] - 0s 1ms/step - loss: 0.7172 - accuracy: 0.5298
Epoch 6/20
102/102 [==============================] - 0s 988us/step - loss: 0.7589 - accuracy: 0.5393
Epoch 7/20
102/102 [==============================] - 0s 1ms/step - loss: 0.7443 - accuracy: 0.5123
Epoch 8/20
102/102 [==============================] - 0s 1ms/step - loss: 0.8563 - accuracy: 0.5279
Epoch 9/20
102/102 [==============================] - 0s 893us/step - loss: 0.8140 - accuracy: 0.5273
Epoch 10/20
102/102 [==============================] - 0s 1ms/step - loss: 0.7714 - accura

# Method 2: Model for Over Sampling

## Bump up the exited count to 6370

In [25]:
count_exited_1_over = count_exited_1.sample(6370,replace=True)
count_exited_1_over.shape

(6370, 13)

In [26]:
count_exited_1_over.Exited.value_counts()

1    6370
Name: Exited, dtype: int64

In [27]:
df_over_input = pd.concat([count_exited_1_over,count_exited_0],axis=0)
df_over_input.shape

(12740, 13)

In [28]:
X= df_over_input.drop(['Exited'],axis='columns')
y= df_over_input.Exited
X.shape, y.shape

((12740, 12), (12740,))

In [29]:
y_pred_over =  ANN(X,y,X_test,y_test,'binary_crossentropy',-1,20)

Epoch 1/20
399/399 [==============================] - 0s 930us/step - loss: 46.0417 - accuracy: 0.5100
Epoch 2/20
399/399 [==============================] - 0s 818us/step - loss: 0.7623 - accuracy: 0.5307
Epoch 3/20
399/399 [==============================] - 0s 769us/step - loss: 0.7842 - accuracy: 0.5357
Epoch 4/20
399/399 [==============================] - 0s 902us/step - loss: 0.7735 - accuracy: 0.5585
Epoch 5/20
399/399 [==============================] - 0s 900us/step - loss: 0.8142 - accuracy: 0.5491
Epoch 6/20
399/399 [==============================] - 0s 976us/step - loss: 0.7376 - accuracy: 0.5712
Epoch 7/20
399/399 [==============================] - 0s 780us/step - loss: 0.8444 - accuracy: 0.5646
Epoch 8/20
399/399 [==============================] - 0s 992us/step - loss: 0.8147 - accuracy: 0.5767
Epoch 9/20
399/399 [==============================] - 0s 814us/step - loss: 0.7240 - accuracy: 0.5948
Epoch 10/20
399/399 [==============================] - 0s 1ms/step - loss: 0.8556

# Method 3: Using SMOTE

## For this we are using the DS created in standard model

In [30]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority', n_jobs=2)

In [31]:
X_sm,y_sm = smote.fit_resample(X_train,y_train)
y_sm.value_counts()

0    6370
1    6370
Name: Exited, dtype: int64

In [32]:
y_pred_sm = ANN(X_sm,y_sm,X_test,y_test,'binary_crossentropy',-1,20)

Epoch 1/20
399/399 [==============================] - 0s 1ms/step - loss: 4.2306 - accuracy: 0.5252
Epoch 2/20
399/399 [==============================] - 0s 955us/step - loss: 0.7892 - accuracy: 0.5521
Epoch 3/20
399/399 [==============================] - 0s 830us/step - loss: 0.7773 - accuracy: 0.5728
Epoch 4/20
399/399 [==============================] - 0s 824us/step - loss: 0.7518 - accuracy: 0.5999
Epoch 5/20
399/399 [==============================] - 0s 771us/step - loss: 0.7320 - accuracy: 0.6194
Epoch 6/20
399/399 [==============================] - 0s 973us/step - loss: 0.7288 - accuracy: 0.6041
Epoch 7/20
399/399 [==============================] - 0s 861us/step - loss: 0.6962 - accuracy: 0.6447
Epoch 8/20
399/399 [==============================] - 0s 978us/step - loss: 0.6820 - accuracy: 0.6459
Epoch 9/20
399/399 [==============================] - 0s 779us/step - loss: 0.6874 - accuracy: 0.6502
Epoch 10/20
399/399 [==============================] - 0s 782us/step - loss: 0.6842 

# Method 4: Ensemble with undersampling using count DS from Method 1

In [33]:
X_train.shape , y_train.shape ,X_test.shape ,y_test.shape

((8000, 12), (8000,), (2000, 12), (2000,))

In [34]:
y_train.value_counts(),  y_test.value_counts()

(0    6370
 1    1630
 Name: Exited, dtype: int64,
 0    1593
 1     407
 Name: Exited, dtype: int64)

In [35]:
count_exited_0.shape, count_exited_1.shape

((6370, 13), (1630, 13))

In [36]:
count_exited_0.Exited.value_counts(), count_exited_1.Exited.value_counts()

(0    6370
 Name: Exited, dtype: int64,
 1    1630
 Name: Exited, dtype: int64)

In [37]:
6370/1630

3.9079754601226995

In [38]:
def get_train_batch(df_majority,df_minority,start,end):
        df_train = pd.concat([df_majority[start:end],df_minority], axis=0)
        X_train = df_train.drop(['Exited'],axis='columns')
        y_train= df_train.Exited
        
        return X_train,y_train
    

In [39]:
X_train,y_train = get_train_batch(count_exited_0,count_exited_1,0,1630)

X_train.shape , y_train.shape

((3260, 12), (3260,))

In [40]:
y_pred_ens_1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1, 20)

Epoch 1/20
102/102 [==============================] - 0s 1ms/step - loss: 20.4231 - accuracy: 0.4957
Epoch 2/20
102/102 [==============================] - 0s 1ms/step - loss: 1.1686 - accuracy: 0.4813
Epoch 3/20
102/102 [==============================] - 0s 987us/step - loss: 1.0100 - accuracy: 0.4954
Epoch 4/20
102/102 [==============================] - 0s 961us/step - loss: 0.9187 - accuracy: 0.5006
Epoch 5/20
102/102 [==============================] - 0s 1ms/step - loss: 1.8966 - accuracy: 0.4975
Epoch 6/20
102/102 [==============================] - 0s 1ms/step - loss: 1.3437 - accuracy: 0.5037
Epoch 7/20
102/102 [==============================] - 0s 1ms/step - loss: 1.1676 - accuracy: 0.5120
Epoch 8/20
102/102 [==============================] - 0s 924us/step - loss: 1.1986 - accuracy: 0.5040
Epoch 9/20
102/102 [==============================] - 0s 981us/step - loss: 1.1449 - accuracy: 0.4963
Epoch 10/20
102/102 [==============================] - 0s 1ms/step - loss: 1.2054 - accurac

In [41]:
X_train,y_train = get_train_batch(count_exited_0,count_exited_1,1630,3260)

X_train.shape , y_train.shape

((3260, 12), (3260,))

In [42]:
y_pred_ens_2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1, 20)

Epoch 1/20
102/102 [==============================] - 0s 1ms/step - loss: 566.3564 - accuracy: 0.5000
Epoch 2/20
102/102 [==============================] - 0s 930us/step - loss: 41.4168 - accuracy: 0.5058
Epoch 3/20
102/102 [==============================] - 0s 908us/step - loss: 0.7499 - accuracy: 0.5095
Epoch 4/20
102/102 [==============================] - 0s 950us/step - loss: 0.7174 - accuracy: 0.5239
Epoch 5/20
102/102 [==============================] - 0s 892us/step - loss: 0.7296 - accuracy: 0.5163
Epoch 6/20
102/102 [==============================] - 0s 1ms/step - loss: 0.7070 - accuracy: 0.5175
Epoch 7/20
102/102 [==============================] - 0s 1ms/step - loss: 0.7145 - accuracy: 0.5178
Epoch 8/20
102/102 [==============================] - 0s 873us/step - loss: 0.7460 - accuracy: 0.5199
Epoch 9/20
102/102 [==============================] - ETA: 0s - loss: 0.7233 - accuracy: 0.52 - 0s 1ms/step - loss: 0.7223 - accuracy: 0.5242
Epoch 10/20
102/102 [========================

In [43]:
X_train,y_train = get_train_batch(count_exited_0,count_exited_1,3260,4890)

X_train.shape , y_train.shape

((3260, 12), (3260,))

In [44]:
y_pred_ens_3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1, 20)

Epoch 1/20
102/102 [==============================] - 0s 823us/step - loss: 174.4783 - accuracy: 0.4859
Epoch 2/20
102/102 [==============================] - 0s 852us/step - loss: 1.5529 - accuracy: 0.5034
Epoch 3/20
102/102 [==============================] - 0s 831us/step - loss: 2.0494 - accuracy: 0.4988
Epoch 4/20
102/102 [==============================] - 0s 973us/step - loss: 3.1638 - accuracy: 0.4923
Epoch 5/20
102/102 [==============================] - 0s 855us/step - loss: 1.4835 - accuracy: 0.5233
Epoch 6/20
102/102 [==============================] - 0s 954us/step - loss: 1.6641 - accuracy: 0.5015
Epoch 7/20
102/102 [==============================] - 0s 962us/step - loss: 1.8190 - accuracy: 0.5206
Epoch 8/20
102/102 [==============================] - 0s 822us/step - loss: 1.6576 - accuracy: 0.5215
Epoch 9/20
102/102 [==============================] - 0s 944us/step - loss: 1.6222 - accuracy: 0.5193
Epoch 10/20
102/102 [==============================] - 0s 1ms/step - loss: 1.391

In [45]:
X_train,y_train = get_train_batch(count_exited_0,count_exited_1,4890,6371)

X_train.shape , y_train.shape

((3110, 12), (3110,))

In [46]:
y_pred_ens_4 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1, 20)

Epoch 1/20
98/98 [==============================] - 0s 901us/step - loss: 27.1696 - accuracy: 0.4939
Epoch 2/20
98/98 [==============================] - 0s 992us/step - loss: 1.1562 - accuracy: 0.5006
Epoch 3/20
98/98 [==============================] - 0s 1ms/step - loss: 0.9208 - accuracy: 0.5254
Epoch 4/20
98/98 [==============================] - 0s 1ms/step - loss: 0.9514 - accuracy: 0.5305
Epoch 5/20
98/98 [==============================] - 0s 1ms/step - loss: 0.8805 - accuracy: 0.5196
Epoch 6/20
98/98 [==============================] - 0s 914us/step - loss: 1.6257 - accuracy: 0.5058
Epoch 7/20
98/98 [==============================] - 0s 824us/step - loss: 1.4842 - accuracy: 0.4965
Epoch 8/20
98/98 [==============================] - 0s 823us/step - loss: 1.1759 - accuracy: 0.5225
Epoch 9/20
98/98 [==============================] - 0s 889us/step - loss: 1.0699 - accuracy: 0.5289
Epoch 10/20
98/98 [==============================] - 0s 830us/step - loss: 0.9642 - accuracy: 0.5386
Epoc

In [47]:
y_pred_final= y_pred_ens_1.copy()

for i in range(len(y_pred_ens_1)):
    n_ones = y_pred_ens_1[i]+y_pred_ens_2[i]+y_pred_ens_3[i]+y_pred_ens_4[i]

    if n_ones >1:
        y_pred_final[i]=1
    else:
        y_pred_final[i]=0

In [48]:
print(classification_report(y_test,y_pred_final))

              precision    recall  f1-score   support

           0       0.91      0.26      0.40      1593
           1       0.24      0.90      0.37       407

    accuracy                           0.39      2000
   macro avg       0.57      0.58      0.39      2000
weighted avg       0.77      0.39      0.40      2000

